In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/DigiKala'

/content/drive/My Drive/DigiKala


In [ ]:
#!unzip 'ML_User_Data.zip'

In [ ]:
!pip install catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from catboost import CatBoostRegressor

***
train data

In [ ]:
df_train = pd.read_csv('train.csv')
print(df_train.shape)
df_train.head()

(900000, 5)


,id,date,seller,item,sales
0,0,2015-05-01,1,1,21
1,1,2015-05-01,2,1,45
2,2,2015-05-01,3,1,28
3,3,2015-05-01,4,1,34
4,4,2015-05-01,5,1,21


In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['dayofmonth'] = df_train['date'].dt.day
df_train['dayofyear'] = df_train['date'].dt.dayofyear
df_train['dayofweek'] = df_train['date'].dt.dayofweek 
df_train['weekofyear'] = df_train['date'].dt.weekofyear
df_train['month'] = df_train['date'].dt.month
df_train['quarter'] = df_train['date'].dt.quarter
df_train['year'] = df_train['date'].dt.year

In [ ]:
df_train.drop(columns={'id', 'date'}, inplace=True)
cols = [col for col in df_train if col != 'sales'] 
df_train = df_train[cols + ['sales']]
df_train.head()

,seller,item,dayofmonth,dayofyear,dayofweek,weekofyear,month,quarter,year,sales
0,1,1,1,121,4,18,5,2,2015,21
1,2,1,1,121,4,18,5,2,2015,45
2,3,1,1,121,4,18,5,2,2015,28
3,4,1,1,121,4,18,5,2,2015,34
4,5,1,1,121,4,18,5,2,2015,21


***
test data

In [ ]:
df_test = pd.read_csv('test.csv')
print(df_test.shape)
df_test.head()

(13000, 4)


,id,date,seller,item
0,0,2020-04-04,1,1
1,1,2020-04-04,2,1
2,2,2020-04-04,3,1
3,3,2020-04-04,4,1
4,4,2020-04-04,5,1


In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['dayofmonth'] = df_test['date'].dt.day
df_test['dayofyear'] = df_test['date'].dt.dayofyear
df_test['dayofweek'] = df_test['date'].dt.dayofweek 
df_test['weekofyear'] = df_test['date'].dt.weekofyear
df_test['month'] = df_test['date'].dt.month
df_test['quarter'] = df_test['date'].dt.quarter
df_test['year'] = df_test['date'].dt.year

In [ ]:
df_test.drop(columns={'id', 'date'}, inplace=True)
df_test.head()

,seller,item,dayofmonth,dayofyear,dayofweek,weekofyear,month,quarter,year
0,1,1,4,95,5,14,4,2,2020
1,2,1,4,95,5,14,4,2,2020
2,3,1,4,95,5,14,4,2,2020
3,4,1,4,95,5,14,4,2,2020
4,5,1,4,95,5,14,4,2,2020


***
Training


In [ ]:
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations = 5000, 
                          eval_metric = 'SMAPE')

In [ ]:
X_train = df_train[cols]
y_train = df_train['sales']
X_test = df_test

In [ ]:
# Fit model
model.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
1:	learn: 45.5417720	total: 321ms	remaining: 13m 21s
2:	learn: 45.1882467	total: 448ms	remaining: 12m 26s
3:	learn: 44.8554981	total: 575ms	remaining: 11m 57s
4:	learn: 44.5401431	total: 704ms	remaining: 11m 43s
5:	learn: 44.2458048	total: 827ms	remaining: 11m 28s
6:	learn: 43.9678730	total: 962ms	remaining: 11m 26s
7:	learn: 43.6967392	total: 1.09s	remaining: 11m 17s
8:	learn: 43.3706700	total: 1.22s	remaining: 11m 16s
9:	learn: 43.0513635	total: 1.34s	remaining: 11m 11s
10:	learn: 42.7948066	total: 1.47s	remaining: 11m 6s
11:	learn: 42.5593618	total: 1.59s	remaining: 11m 2s
12:	learn: 42.3410544	total: 1.72s	remaining: 11m
13:	learn: 42.0903481	total: 1.83s	remaining: 10m 50s
14:	learn: 41.8345502	total: 1.96s	remaining: 10m 50s
15:	learn: 41.5925743	total: 2.08s	remaining: 10m 48s
16:	learn: 41.3947890	total: 2.21s	remaining: 10m 48s
17:	learn: 41.1671711	total: 2.33s	remaining: 10m 46s
18:	learn: 40.9630333	total: 2.45s	remaining: 

In [ ]:
# Get predictions
preds = model.predict(X_test)

In [ ]:
df_pred = pd.DataFrame(preds, columns = ['sales'])
df_pred.head()

,sales
0,36.151955
1,52.082814
2,46.179883
3,42.527191
4,30.876284


In [ ]:
df_pred.to_csv('answer_style.csv', index_label=['id'], index = True)